In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
x = df[['Pclass','SibSp','Parch','Fare']]
y = df.Survived

In [7]:
y.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [8]:
x.head()

,Pclass,SibSp,Parch,Fare
0,3,1,0,7.2500
1,1,1,0,71.2833
2,3,0,0,7.9250
3,1,1,0,53.1000
4,3,0,0,8.0500


In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25)

In [10]:
x_train.columns

Index(['Pclass', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [11]:
s_col = ['Fare']

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train.loc[:,s_col] = sc.fit_transform(x_train[s_col])

In [13]:
x_test.loc[:,s_col] = sc.fit_transform(x_test[s_col])

In [14]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
svcr = SVC(kernel='rbf')
svcr.fit(x_train,y_train)
y_pred=svcr.predict(x_test)
accuracy=accuracy_score(y_test,y_pred)
print('{:.2f}'.format(accuracy))

0.68


In [15]:
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits = 5)
score = cross_val_score(svcr,x,y,cv=kf)
print ("Average cross val score for {} : {:.3f}".format(svcr,score.mean()))

Average cross val score for SVC() : 0.666


In [16]:
parameters ={'C': [0.1, 1, 10, 100, 1000],
             'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
             'kernel': ['rbf']}

In [17]:
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(estimator = SVC(), param_distributions=parameters,verbose=1, n_jobs=-1,
                            n_iter=50)
random_result = random_search.fit(x_train, y_train)
print('Best Score: ', random_result.best_score_*100)
print('Best Params: ', random_result.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best Score:  73.34754797441364
Best Params:  {'kernel': 'rbf', 'gamma': 0.1, 'C': 100}


In [18]:
# hyperopt

In [20]:
pip install hyperopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials

In [23]:
space = {
    "C": hp.choice("C", [0.1, 1, 10, 100, 1000]),
    "gamma": hp.choice("gamma", [1, 0.1, 0.01, 0.001, 0.0001])
}

In [24]:
def hyperparameter_tuning(params):
    svcr = SVC(**params)
    acc = cross_val_score(svcr, x, y,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [25]:
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=50, 
    trials=trials
)

print("Best: {}".format(best))

100%|██████████| 50/50 [00:27<00:00,  1.80it/s, best loss: -0.6993032452451196]
Best: {'C': 3, 'gamma': 1}
